<a href="https://colab.research.google.com/github/BenjaminDKLuong/Colab_Notes/blob/master/Learning_rates_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import math

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint, LearningRateScheduler, Callback
from keras import backend as K

from keras.datasets import cifar10


Using TensorFlow backend.


In [2]:
SEED =2017

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
validation_split = 0.1
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=validation_split, random_state=SEED)

X_train = X_train.astype('float32')
X_train /=255.
X_val = X_val.astype('float32')
X_val /=255.
X_test = X_test.astype('float32')
X_test /=255.

n_classes = 10
y_train = to_categorical(y_train, n_classes)
y_val = to_categorical(y_val, n_classes)
y_test = to_categorical(y_test, n_classes)

170500096/170498071 [==============================] - 43s 0us/step


In [0]:
learning_rate_schedule = {0: '0.1', 10: '0.01', 25: '0.0025'}

class get_learning_rate(Callback):
    def on_epoch_end(self, epoch, logs={}):
        optimizer = self.model.optimizer
        if epoch in learning_rate_schedule:
            K.set_value(optimizer.lr, learning_rate_schedule[epoch])
        lr = K.eval(optimizer.lr)
        print('\nlr: {:.4f}'.format(lr))

In [0]:
callbacks =[EarlyStopping(monitor='val_acc', patience=5, verbose=2),
            ModelCheckpoint('checkpoints/{epoch:02d}.h5', 
            save_best_only=True),
            TensorBoard('~/notebooks/logs-lrscheduler'),
            get_learning_rate()
            ]

In [5]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation('softmax'))

optimizer = SGD()
model.compile(loss='categorical_crossentropy',optimizer=optimizer, metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
import os

dirName = "/content/checkpoints"
try:
# Create target Directory
	os.mkdir(dirName)
	print("Directory " , dirName ,  " Created ") 
except FileExistsError:
print("Directory " , dirName ,  " already exists")

In [10]:
n_epochs = 20
batch_size = 128

history = model.fit(X_train, y_train, epochs=n_epochs, batch_size=batch_size, 
          validation_data=[X_val, y_val],
          verbose = 1, callbacks=callbacks)

Train on 45000 samples, validate on 5000 samples
Epoch 1/20
45000/45000 [==============================] - 4s 97us/step - loss: 2.0470 - acc: 0.2464 - val_loss: 2.0119 - val_acc: 0.2624

lr: 0.1000
Epoch 2/20
45000/45000 [==============================] - 4s 94us/step - loss: 2.1712 - acc: 0.1968 - val_loss: 2.1051 - val_acc: 0.2358

lr: 0.1000
Epoch 3/20
45000/45000 [==============================] - 4s 95us/step - loss: 1.8296 - acc: 0.3475 - val_loss: 1.7004 - val_acc: 0.3926

lr: 0.1000
Epoch 4/20
45000/45000 [==============================] - 4s 94us/step - loss: 1.5838 - acc: 0.4309 - val_loss: 1.4214 - val_acc: 0.4882

lr: 0.1000
Epoch 5/20
45000/45000 [==============================] - 4s 95us/step - loss: 1.4552 - acc: 0.4774 - val_loss: 1.3624 - val_acc: 0.5114

lr: 0.1000
Epoch 6/20
45000/45000 [==============================] - 4s 96us/step - loss: 1.3679 - acc: 0.5083 - val_loss: 1.2304 - val_acc: 0.5626

lr: 0.1000
Epoch 7/20
45000/45000 [==============================] -